# 1. Slack 설치

In [1]:
# slack_sdk 패키지 설치

!pip install slack_sdk

---
# 2. Slack_sdk 초기설정

- slack_sdk 패키지 import
- WebClient 연결

In [15]:
# slack_sdk 패키지에서 필요 정보 불러오기

from slack_sdk import WebClient
import os

slack_key = '~'
channel_id = '~'


client = WebClient(token=os.environ.get(slack_key))
client.token = slack_key

---
# 3. post_message 함수로 메세지 테스트 메세지 보내보기

In [9]:
# 보낼 메세지와 보낼 채널 정보를 받아오는 post_message 함수

def post_message(message,channel_id = None):
    client.chat_postMessage(
        channel=channel_id,
        text=message,
        mrkdwn=False
    )


In [10]:
# 테스트 메세지를 보내본다
post_message('test message',channel_id=channel_id)

---
# 4. HantuStock에 slack api 추가하기

- Slack 클래스 생성
- HantuStock이 Slack 클래스를 상속하도록 개발

In [12]:
# Slack 클래스 생성
class Slack:
    def activate_slack(self,slack_key):
        
        self.client = WebClient(token=os.environ.get(slack_key))
        self.client.token = slack_key

    def post_message(self,message,channel_id = None):
        self.client.chat_postMessage(
            channel=channel_id,
            text=message,
            mrkdwn=False
        )


In [13]:
# HantuStock이 Slack을 상속받게 하려면 아래와 같이 괄호 안에 Slack 함수를 넣어줌
class HantuTest(Slack):
    def some_codes(self):
        pass

In [17]:
# HantuStock이 Slack을 상속받게 만든 후, Slack 클래스의 activate_slack를 실행
ht = HantuTest()

In [ ]:
ht.activate_slack(slack_key=slack_key)
ht.post_message('test message for HantuTest',channel_id=channel_id)

---
# 5. 사용 예시

In [ ]:
# HantuStock 패키지 불러오기

import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

api_key = config['hantu']['api_key']
secret_key = config['hantu']['secret_key']
account_id = config['hantu']['account_id']

from HantuStock import HantuStock

ht = HantuStock(api_key=api_key,secret_key=secret_key,account_id=account_id)
ht.activate_slack(slack_key=slack_key)

In [ ]:
# 슬랙 활용 방법 예시 - 매도주문 이후 알림

ticker_to_sell = ['005930','000660']

while True:
    current_time = datetime.now()

    if current_time.hour == 15 and current_time.minute == 20:
        holdings = ht.get_holding_stock()
        
        for tkr in ticker_to_sell:
            ht.ask(tkr,'market',holdings[tkr],'STOCK')
            # 매도주문 직후 post_message로 매도주문의 상세정보를 slack으로 보내줌
            ht.post_message('{} 매도주문!'.format(tkr))

    time.sleep(1)
